# Model Making Kitchen

Create three candidate models for the ensemble modeling challenge

## Setting: New York State, Timepoint 2: July 19, 2021

### Gather population values and initial conditions for this setting
Population values from [Census Reporter - New York](https://censusreporter.org/profiles/04000US36-new-york/)

Initial conditions estimated from [COVID-19 Forecast Hub](https://github.com/reichlab/covid19-forecast-hub/tree/master/data-truth)

In [1]:
# Population
NYS_total_pop = 19_677_152
pop_frac = [0.202, 0.617, 0.181]
NYS_young_pop = NYS_total_pop*pop_frac[0] # under 18
NYS_middle_pop = NYS_total_pop*pop_frac[1] # ages 18 - 64
NYS_old_pop = NYS_total_pop*pop_frac[2] # ages 65+

# Initial conditions `{age}_initials = [E0, I0, R0, H0, D0]`
cumul_cases0 = 2_102_869.0
cumul_hosp0 = 136_862.0
all_initials = [4_000.0, 4_000.0, 2_102_000.0, 700.0, 53_123.0]
y_initials = [i * pop_frac[0] for i in all_initials]
m_initials = [i * pop_frac[1] for i in all_initials]
o_initials = [i * pop_frac[2] for i in all_initials]

### Load dependencies

In [2]:
import os
import json
import sympy

from mira.metamodel import *
from mira.metamodel.ops import stratify
from mira.examples.concepts import susceptible, exposed, infected, recovered
from mira.modeling import Model
from mira.modeling.amr.petrinet import AMRPetriNetModel, template_model_to_petrinet_json
from mira.sources.amr.petrinet import template_model_from_amr_json
from mira.metamodel.io import model_to_json_file, model_from_json_file

# Model 1: Age-structured SEIRHD model
Adapted from the Scenario 3 model for the 18-month evaluation, uses the same contact matrix

### (1) Define units

In [3]:
person_units = lambda: Unit(expression=sympy.Symbol('person'))
day_units = lambda: Unit(expression=sympy.Symbol('day'))
per_day_units = lambda: Unit(expression=1/sympy.Symbol('day'))
dimensionless_units = lambda: Unit(expression=sympy.Integer('1'))
per_day_per_person_units = lambda: Unit(expression=1/(sympy.Symbol('day')*sympy.Symbol('person')))

### (2) Define and stratify model concepts

In [4]:
_susceptible = Concept(name='S', units=person_units(), identifiers={'ido': '0000514'})
_exposed = Concept(name='E', units=person_units(), identifiers={'apollosv': '00000154'})
_infected = Concept(name='I', units=person_units(), identifiers={'ido': '0000511'})
_recovered = Concept(name='R', units=person_units(), identifiers={'ido': '0000592'})
_hospitalized = Concept(name="H", units=person_units(), identifiers={"ido": "0000511"},
                        context={"property": "ncit:C25179"})
_deceased = Concept(name="D", units=person_units(), identifiers={"ncit": "C28554"})

c = {
    'S_y': _susceptible.with_context(status="young"),
    'S_m': _susceptible.with_context(status="middle"),
    'S_o': _susceptible.with_context(status="old"),
    
    'E_y': _exposed.with_context(status="young"),
    'E_m': _exposed.with_context(status="middle"),
    'E_o': _exposed.with_context(status="old"),
    
    'I_y': _infected.with_context(status="young"),
    'I_m': _infected.with_context(status="middle"),
    'I_o': _infected.with_context(status="old"),
    
    'R_y': _recovered.with_context(status="young"),
    'R_m': _recovered.with_context(status="middle"),
    'R_o': _recovered.with_context(status="old"),
    
    'H_y': _recovered.with_context(status="young"),
    'H_m': _recovered.with_context(status="middle"),
    'H_o': _recovered.with_context(status="old"),
    
    'D_y': _recovered.with_context(status="young"),
    'D_m': _recovered.with_context(status="middle"),
    'D_o': _recovered.with_context(status="old"),
    
    "Cumulative_cases": Concept(name="Cumulative_cases", units=person_units()),
    "Cumulative_hosp": Concept(name="Cumulative_hosp", units=person_units())
}

for concept in c:
    c[concept].name = concept

### (3) Define parameters with uncertainty
Parameter values estimated from [CDC COVID-19 Pandemic Planning Scenarios](https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html)

Contact matrix from [18-month Epi Eval Scenario 3 model](https://github.com/DARPA-ASKEM/program-milestones/blob/main/18-month-milestone/evaluation/Epi%20Use%20Case/ASKEM_18Month_Epi_Evaluation_Scenarios_03.22.2024_FINAL.pdf) 

In [5]:
parameters = {
    'beta': Parameter(name='beta', value=sympy.Float(0.0005), units=per_day_units(),
                      distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.0001,
                                                            'maximum': 0.001})),  # Transmission rate
    'N': Parameter(name='total_population', value=sympy.Float(NYS_total_pop), units=person_units()),  # Total population
    
    'r_EI': Parameter(name='r_EI', value=sympy.Float(0.16), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.1,
                                                            'maximum': 0.25})),  # Rate of progressing E -> I
    
    'r_IR_y': Parameter(name='r_IR_y', value=sympy.Float(0.475), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.4,
                                                            'maximum': 0.5})),  # Rate of progressing I_y -> R_y
    'r_IR_m': Parameter(name='r_IR_m', value=sympy.Float(0.15), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.1,
                                                        'maximum': 0.2})),  # Rate of progressing I_m -> R_m
    'r_IR_o': Parameter(name='r_IR_o', value=sympy.Float(0.2125), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.25})),  # Rate of progressing I_o -> R_o

    'r_IH_y': Parameter(name='r_IH_y', value=sympy.Float(0.025), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.02,
                                                            'maximum': 0.03})),  # Rate of progressing I_y -> H_y
    'r_IH_m': Parameter(name='r_IH_m', value=sympy.Float(0.016), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.01,
                                                        'maximum': 0.1})),  # Rate of progressing I_m -> H_m
    'r_IH_o': Parameter(name='r_IH_o', value=sympy.Float(0.0375), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.02,
                                                        'maximum': 0.04})),  # Rate of progressing I_o -> H_o
    
    'r_HR_y': Parameter(name='r_HR_y', value=sympy.Float(0.331), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.3,
                                                            'maximum': 0.4})),  # Rate of progressing H_y -> R_y
    'r_HR_m': Parameter(name='r_HR_m', value=sympy.Float(0.1583), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.1,
                                                        'maximum': 0.2})),  # Rate of progressing H_m -> R_m
    'r_HR_o': Parameter(name='r_HR_o', value=sympy.Float(0.116), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.09,
                                                        'maximum': 0.13})),  # Rate of progressing H_o -> R_o
    
    'r_HD_y': Parameter(name='r_HD_y', value=sympy.Float(0.002), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.001,
                                                            'maximum': 0.003})),  # Rate of progressing H_y -> D_y
    'r_HD_m': Parameter(name='r_HD_m', value=sympy.Float(0.0083), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.008,
                                                        'maximum': 0.009})),  # Rate of progressing D_m -> D_m
    'r_HD_o': Parameter(name='r_HD_o', value=sympy.Float(0.0268), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.02,
                                                        'maximum': 0.035})),  # Rate of progressing D_o -> D_o
    
    'Myy': Parameter(name='Myy', value=sympy.Float(38.62), units=per_day_units()),  # Contact rate young -> young
    'Mym': Parameter(name='Mym', value=sympy.Float(20.56), units=per_day_units()),  # Contact rate young -> middle
    'Myo': Parameter(name='Myo', value=sympy.Float(6.12), units=per_day_units()),  # Contact rate young -> old
    'Mmy': Parameter(name='Mmy', value=sympy.Float(20.56), units=per_day_units()),  # Contact rate middle -> young
    'Mmm': Parameter(name='Mmm', value=sympy.Float(28.22), units=per_day_units()),  # Contact rate middle -> middle
    'Mmo': Parameter(name='Mmo', value=sympy.Float(11.6), units=per_day_units()),  # Contact rate middle -> old
    'Moy': Parameter(name='Moy', value=sympy.Float(6.12), units=per_day_units()),  # Contact rate old -> young
    'Mom': Parameter(name='Mom', value=sympy.Float(11.6), units=per_day_units()),  # Contact rate old -> middle
    'Moo': Parameter(name='Moo', value=sympy.Float(20.01), units=per_day_units()),  # Contact rate old -> old
}

### (4) Define `SymPy` variables

In [6]:
S_y, S_m, S_o, E_y, E_m, E_o, I_y, I_m, I_o, R_y, R_m, R_o, H_y, H_m, H_o, D_y, D_m, D_o, Cumulative_cases, Cumulative_hosp, beta, N, r_EI, r_IR_y, r_IR_m, r_IR_o, r_IH_y, r_IH_m, r_IH_o, r_HR_y, r_HR_m, r_HR_o, r_HD_y, r_HD_m, r_HD_o, Myy, Mym, Myo, Mmy, Mmm, Mmo, Moy, Mom, Moo = \
    sympy.symbols(
        'S_y S_m S_o E_y E_m E_o I_y I_m I_o R_y R_m R_o H_y H_m H_o D_y D_m D_o Cumulative_cases Cumulative_hosp beta N r_EI r_IR_y r_IR_m r_IR_o r_IH_y r_IH_m r_IH_o r_HR_y r_HR_m r_HR_o r_HD_y r_HD_m r_HD_o Myy Mym Myo Mmy Mmm Mmo Moy Mom Moo'
    )

### (5) Set initial conditions

In [7]:
initials = {
    "S_y": Initial(concept=c["S_y"], expression=NYS_young_pop - sum(y_initials)),
    "S_m": Initial(concept=c["S_m"], expression=NYS_middle_pop - sum(m_initials)),
    "S_o": Initial(concept=c["S_o"], expression=NYS_old_pop - sum(o_initials)),
    "E_y": Initial(concept=c["E_y"], expression=y_initials[0]),
    "E_m": Initial(concept=c["E_m"], expression=m_initials[0]),
    "E_o": Initial(concept=c["E_o"], expression=o_initials[0]),
    "I_y": Initial(concept=c["I_y"], expression=y_initials[1]),
    "I_m": Initial(concept=c["I_m"], expression=m_initials[1]),
    "I_o": Initial(concept=c["I_o"], expression=o_initials[1]),
    "R_y": Initial(concept=c["R_y"], expression=y_initials[2]),
    "R_m": Initial(concept=c["R_m"], expression=m_initials[2]),
    "R_o": Initial(concept=c["R_o"], expression=o_initials[2]),
    'H_y': Initial(concept=c["H_y"], expression=y_initials[3]),
    'H_m': Initial(concept=c["H_m"], expression=m_initials[3]),
    'H_o': Initial(concept=c["H_o"], expression=o_initials[3]),
    'D_y': Initial(concept=c["D_y"], expression=y_initials[4]),
    'D_m': Initial(concept=c["D_m"], expression=m_initials[4]),
    'D_o': Initial(concept=c["D_o"], expression=o_initials[4]),
    "Cumulative_cases": Initial(concept=c["Cumulative_cases"], expression=cumul_cases0),
    "Cumulative_hosp": Initial(concept=c["Cumulative_hosp"], expression=cumul_hosp0)
}

### (6) Define templates

In [8]:
##### S -> E
# Sy -> Ey by Iy
syeyiy = ControlledConversion(
    subject=c['S_y'],
    outcome=c['E_y'],
    controller=c['I_y'],
    rate_law=beta*S_y*(Myy*I_y) / N
)
# Sy -> Ey by Im
syeyim = ControlledConversion(
    subject=c['S_y'],
    outcome=c['E_y'],
    controller=c['I_m'],
    rate_law=beta*S_y*(Mym*I_m) / N
)
# Sy -> Ey by Io
syeyio = ControlledConversion(
    subject=c['S_y'],
    outcome=c['E_y'],
    controller=c['I_o'],
    rate_law=beta*S_y*(Myo*I_o) / N
)

# Sm -> Em by Iy
smemiy = ControlledConversion(
    subject=c['S_m'],
    outcome=c['E_m'],
    controller=c['I_y'],
    rate_law=beta*S_m*(Mmy*I_y) / N
)
# Sm -> Em by Im
smemim = ControlledConversion(
    subject=c['S_m'],
    outcome=c['E_m'],
    controller=c['I_m'],
    rate_law=beta*S_m*(Mmm*I_m) / N
)
# Sm -> Em by Io
smemio = ControlledConversion(
    subject=c['S_m'],
    outcome=c['E_m'],
    controller=c['I_o'],
    rate_law=beta*S_m*(Mmo*I_o) / N
)

# So -> Eo by Iy
soeoiy = ControlledConversion(
    subject=c['S_o'],
    outcome=c['E_o'],
    controller=c['I_y'],
    rate_law=beta*S_o*(Moy*I_y) / N
)
# So -> Eo by Im
soeoim = ControlledConversion(
    subject=c['S_o'],
    outcome=c['E_o'],
    controller=c['I_m'],
    rate_law=beta*S_o*(Mom*I_m) / N
)
# So -> Eo by Io
soeoio = ControlledConversion(
    subject=c['S_o'],
    outcome=c['E_o'],
    controller=c['I_o'],
    rate_law=beta*S_o*(Moo*I_o) / N
)


#### E -> I
# Ey -> Iy
eyiy = NaturalConversion(
    subject=c['E_y'],
    outcome=c['I_y'],
    rate_law=r_EI*E_y
)
# Em -> Im
emim = NaturalConversion(
    subject=c['E_m'],
    outcome=c['I_m'],
    rate_law=r_EI*E_m
)
# Eo -> Io
eoio = NaturalConversion(
    subject=c['E_o'],
    outcome=c['I_o'],
    rate_law=r_EI*E_o
)


#### I -> R
# Iy -> Ry
iyry = NaturalConversion(
    subject=c['I_y'],
    outcome=c['R_y'],
    rate_law=r_IR_y*I_y
)
# Im -> Rm
imrm = NaturalConversion(
    subject=c['I_m'],
    outcome=c['R_m'],
    rate_law=r_IR_m*I_m
)
# Io -> Ro
ioro = NaturalConversion(
    subject=c['I_o'],
    outcome=c['R_o'],
    rate_law=r_IR_m*I_o
)


#### I -> H
# Iy -> Hy
iyhy = NaturalConversion(
    subject=c['I_y'],
    outcome=c['H_y'],
    rate_law=r_IH_y*I_y
)
# Im -> Hm
imhm = NaturalConversion(
    subject=c['I_m'],
    outcome=c['H_m'],
    rate_law=r_IH_m*I_m
)
# Io -> Ho
ioho = NaturalConversion(
    subject=c['I_o'],
    outcome=c['H_o'],
    rate_law=r_IH_o*I_o
)


#### H -> R
# Hy -> Ry
hyry = NaturalConversion(
    subject=c['H_y'],
    outcome=c['R_y'],
    rate_law=r_HR_y*H_y
)
# Hm -> Rm
hmrm = NaturalConversion(
    subject=c['H_m'],
    outcome=c['R_m'],
    rate_law=r_HR_m*H_m
)
# Ho -> Ro
horo = NaturalConversion(
    subject=c['H_o'],
    outcome=c['R_o'],
    rate_law=r_HR_o*H_o
)


#### H -> D
# Hy -> Dy
hydy = NaturalConversion(
    subject=c['H_y'],
    outcome=c['D_y'],
    rate_law=r_HD_y*H_y
)
# Hm -> Dm
hmdm = NaturalConversion(
    subject=c['H_m'],
    outcome=c['D_m'],
    rate_law=r_HD_m*H_m
)
# Ho -> Do
hodo = NaturalConversion(
    subject=c['H_o'],
    outcome=c['D_o'],
    rate_law=r_HD_o*H_o
)


### Cumulative Cases
# Ey -> Iy
ccy = ControlledProduction(
    controller=c['E_y'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_y
)
# Em -> Im
ccm = ControlledProduction(
    controller=c['E_m'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_m
)
# Eo -> Io
cco = ControlledProduction(
    controller=c['E_o'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_o
)


### Cumulative Hospitalizations
# Iy -> Hy
chy = NaturalConversion(
    subject=c['I_y'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_y*I_y
)
# Im -> Hm
chm = NaturalConversion(
    subject=c['I_m'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_m*I_m
)
# Io -> Ho
cho = NaturalConversion(
    subject=c['I_o'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_o*I_o
)

### (7) Define observables (including for cumulative cases, hospitalizations, and deaths)

In [9]:
observables_seir = {
    'susceptible': Observable(name='susceptible', expression=S_y+S_m+S_o),
    'exposed': Observable(name='exposed', expression=E_y+E_m+E_o),
    'infected': Observable(name='infected', expression=I_y+I_m+I_o),
    'recovered': Observable(name='recovered', expression=R_y+R_m+R_o),
    'hospitalized': Observable(name='hospitalized', expression=H_y+H_m+H_o),
    'deceased': Observable(name='deceased', expression=D_y+D_m+D_o),
    'all_cases': Observable(name='all_cases', expression=Cumulative_cases),
    'all_hosp': Observable(name='all_hosp', expression=Cumulative_hosp)
}

### (8) Define template model and save as a petrinet AMR

In [10]:
seir_model = TemplateModel(
    templates=[
        syeyiy,
        syeyim,
        syeyio,
        smemiy,
        smemim,
        smemio,
        soeoiy,
        soeoim,
        soeoio,
        eyiy,
        emim,
        eoio,
        iyry,
        imrm,
        ioro,
        iyhy,
        imhm,
        ioho,
        hyry,
        hmrm,
        horo,
        hydy,
        hmdm,
        hodo,
        ccy,
        ccm,
        cco,
        chy,
        chm,
        cho
    ],
    parameters=parameters,
    initials=initials,
    time=Time(name='t', units=day_units()),
    observables=observables_seir,
    annotations=Annotations(name='SEIRHD age-structured model')
)

# Save as JSON
with open("SEIRHD_age_structured_petrinet.json", 'w') as fh:
    json.dump(template_model_to_petrinet_json(seir_model), fh, indent=1)

# Model 2: SEIRHD model with vaccination and two strains

### (1) Define units (same as above)

### (2) Define and stratify model concepts

In [11]:
_susceptible = Concept(name='S', units=person_units(), identifiers={'ido': '0000514'})
_exposed = Concept(name='E', units=person_units(), identifiers={'apollosv': '00000154'})
_infected = Concept(name='I', units=person_units(), identifiers={'ido': '0000511'})
_recovered = Concept(name='R', units=person_units(), identifiers={'ido': '0000592'})
_hospitalized = Concept(name="H", units=person_units(), identifiers={"ido": "0000511"},
                        context={"property": "ncit:C25179"})
_deceased = Concept(name="D", units=person_units(), identifiers={"ncit": "C28554"})

c = {
    'S_u': _susceptible.with_context(status="unvaccinated"),
    'S_v': _susceptible.with_context(status="vaccinated"),
    
    'E_u_alpha': _exposed.with_context(status="unvaccinated_alpha"),
    'E_v_alpha': _exposed.with_context(status="vaccinated_alpha"),
    'E_u_delta': _exposed.with_context(status="unvaccinated_delta"),
    'E_v_delta': _exposed.with_context(status="vaccinated_delta"),
    
    'I_u_alpha': _infected.with_context(status="unvaccinated_alpha"),
    'I_v_alpha': _infected.with_context(status="vaccinated_alpha"),
    'I_u_delta': _infected.with_context(status="unvaccinated_delta"),
    'I_v_delta': _infected.with_context(status="vaccinated_delta"),
    
    'R_u': _recovered.with_context(status="unvaccinated"),
    'R_v': _recovered.with_context(status="vaccinated"),
    
    'H_u': _recovered.with_context(status="unvaccinated"),
    'H_v': _recovered.with_context(status="vaccinated"),
    
    'D_u': _recovered.with_context(status="unvaccinated"),
    'D_v': _recovered.with_context(status="vaccinated"),
    
    "Cumulative_cases": Concept(name="Cumulative_cases", units=person_units()),
    "Cumulative_hosp": Concept(name="Cumulative_hosp", units=person_units())
}

for concept in c:
    c[concept].name = concept

### (3) Define parameters with uncertainty

In [12]:
parameters = {
    'beta': Parameter(name='beta', value=sympy.Float(0.2), units=per_day_units(),
                      distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.05,
                                                            'maximum': 0.8})),  # Transmission rate
    'N': Parameter(name='total_population', value=sympy.Float(NYS_total_pop), units=person_units()),  # Total population
    
    'r_EI': Parameter(name='r_EI', value=sympy.Float(0.16), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.1,
                                                            'maximum': 0.25})),  # Rate of progressing E -> I
    
    'r_IR_ua': Parameter(name='r_IR_ua', value=sympy.Float(0.15), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.1,
                                                            'maximum': 0.2})),  # Rate of progressing I_ua -> R_u
    'r_IR_va': Parameter(name='r_IR_va', value=sympy.Float(0.162), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.2})),  # Rate of progressing I_va -> R_v
    'r_IR_ud': Parameter(name='r_IR_ud', value=sympy.Float(0.155), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.2})),  # Rate of progressing I_ud -> R_u
    'r_IR_vd': Parameter(name='r_IR_vd', value=sympy.Float(0.162), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.2})),  # Rate of progressing I_vd -> R_v

    'r_IH_ua': Parameter(name='r_IH_ua', value=sympy.Float(0.016), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.012,
                                                            'maximum': 0.03})),  # Rate of progressing I_ua -> H_u
    'r_IH_va': Parameter(name='r_IH_va', value=sympy.Float(0.005), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.002,
                                                        'maximum': 0.01})),  # Rate of progressing I_va -> H_v
    'r_IH_ud': Parameter(name='r_IH_ud', value=sympy.Float(0.015), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.012,
                                                        'maximum': 0.03})),  # Rate of progressing I_ud -> H_u
    'r_IH_vd': Parameter(name='r_IH_vd', value=sympy.Float(0.005), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.001,
                                                        'maximum': 0.04})),  # Rate of progressing I_vd -> H_v
    
    'r_HR_u': Parameter(name='r_HR_y', value=sympy.Float(0.15), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.12,
                                                            'maximum': 0.2})),  # Rate of progressing H_u -> R_u
    'r_HR_v': Parameter(name='r_HR_m', value=sympy.Float(0.16), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.1,
                                                        'maximum': 0.2})),  # Rate of progressing H_v -> R_v
    
    'r_HD_u': Parameter(name='r_HD_u', value=sympy.Float(0.016), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.01,
                                                            'maximum': 0.02})),  # Rate of progressing H_u -> D_u
    'r_HD_v': Parameter(name='r_HD_m', value=sympy.Float(0.006), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.005,
                                                        'maximum': 0.009})),  # Rate of progressing H_v -> D_v
    
    'Mua': Parameter(name='Mua', value=sympy.Float(1.0), units=per_day_units()),  # Transmission multiplier: S_u and I_alpha
    'Mva': Parameter(name='Mva', value=sympy.Float(0.6), units=per_day_units()),  # Transmission multiplier: S_v and I_alpha
    'Mud': Parameter(name='Mud', value=sympy.Float(1.3), units=per_day_units()),  # Transmission multiplier: S_u and I_delta
    'Mvd': Parameter(name='Mvd', value=sympy.Float(0.6), units=per_day_units()),  # Transmission multiplier: S_v and I_delta
}

### (4) Define `SymPy` variables

In [13]:
S_u, S_v, E_u_alpha, E_v_alpha, E_u_delta, E_v_delta, I_u_alpha, I_v_alpha, I_u_delta, I_v_delta, R_u, R_v, H_u, H_v, D_u, D_v, Cumulative_cases, Cumulative_hosp, beta, N, r_EI, r_IR_ua, r_IR_va, r_IR_ud, r_IR_vd, r_IH_ua, r_IH_va, r_IH_ud, r_IH_vd, r_HR_u, r_HR_v, r_HD_u, r_HD_v, Mua, Mva, Mud, Mvd = \
    sympy.symbols(
        'S_u S_v E_u_alpha E_v_alpha E_u_delta E_v_delta I_u_alpha I_v_alpha I_u_delta I_v_delta R_u R_v H_u H_v D_u D_v Cumulative_cases Cumulative_hosp beta N r_EI r_IR_ua r_IR_va r_IR_ud r_IR_vd r_IH_ua r_IH_va r_IH_ud r_IH_vd r_HR_u r_HR_v r_HD_u r_HD_v Mua Mva Mud Mvd'
    )

### (5) Set initial conditions

Vaccination data from https://usafacts.org/visualizations/covid-vaccine-tracker-states/state/new-york/

In [14]:
NYS_vacc_pop = 0.6*NYS_total_pop
NYS_unvacc_pop = NYS_total_pop - NYS_vacc_pop
initials = {
    "S_u": Initial(concept=c["S_u"], expression=NYS_unvacc_pop - (sum(all_initials)/2)),
    "S_v": Initial(concept=c["S_v"], expression=NYS_vacc_pop - (sum(all_initials)/2)),
    "E_u_alpha": Initial(concept=c["E_u_alpha"], expression=all_initials[0]/4),
    "E_v_alpha": Initial(concept=c["E_v_alpha"], expression=all_initials[0]/4),
    "E_u_delta": Initial(concept=c["E_u_delta"], expression=all_initials[0]/4),
    "E_v_delta": Initial(concept=c["E_v_delta"], expression=all_initials[0]/4),
    "I_u_alpha": Initial(concept=c["I_u_alpha"], expression=all_initials[1]/4),
    "I_v_alpha": Initial(concept=c["I_v_alpha"], expression=all_initials[1]/4),
    "I_u_delta": Initial(concept=c["I_u_delta"], expression=all_initials[1]/4),
    "I_v_delta": Initial(concept=c["I_v_delta"], expression=all_initials[1]/4),
    "R_u": Initial(concept=c["R_u"], expression=all_initials[2]/2),
    "R_v": Initial(concept=c["R_v"], expression=all_initials[2]/2),
    'H_u': Initial(concept=c["H_u"], expression=all_initials[3]/2),
    'H_v': Initial(concept=c["H_v"], expression=all_initials[3]/2),
    'D_u': Initial(concept=c["D_u"], expression=all_initials[4]/2),
    'D_v': Initial(concept=c["D_v"], expression=all_initials[4]/2),
    "Cumulative_cases": Initial(concept=c["Cumulative_cases"], expression=cumul_cases0),
    "Cumulative_hosp": Initial(concept=c["Cumulative_hosp"], expression=cumul_hosp0)
}

### (6) Define templates

In [15]:
##### S -> E
# Su -> Eua by Iua
suua = ControlledConversion(
    subject=c['S_u'],
    outcome=c['E_u_alpha'],
    controller=c['I_u_alpha'],
    rate_law=beta*S_u*(Mua*I_u_alpha) / N
)
# Sv -> Eva by Iua
svua = ControlledConversion(
    subject=c['S_v'],
    outcome=c['E_v_alpha'],
    controller=c['I_u_alpha'],
    rate_law=beta*S_v*(Mva*I_u_alpha) / N
)
# Su -> Eua by Iva
suva = ControlledConversion(
    subject=c['S_u'],
    outcome=c['E_u_alpha'],
    controller=c['I_v_alpha'],
    rate_law=beta*S_u*(Mua*I_v_alpha) / N
)
# Sv -> Eva by Iva
svva = ControlledConversion(
    subject=c['S_v'],
    outcome=c['E_v_alpha'],
    controller=c['I_v_alpha'],
    rate_law=beta*S_v*(Mva*I_v_alpha) / N
)


# Su -> Eud by Iud
suud = ControlledConversion(
    subject=c['S_u'],
    outcome=c['E_u_delta'],
    controller=c['I_u_delta'],
    rate_law=beta*S_u*(Mud*I_u_delta) / N
)
# Sv -> Evd by Iud
svud = ControlledConversion(
    subject=c['S_v'],
    outcome=c['E_v_delta'],
    controller=c['I_u_delta'],
    rate_law=beta*S_v*(Mvd*I_u_delta) / N
)
# Su -> Eud by Ivd
suvd = ControlledConversion(
    subject=c['S_u'],
    outcome=c['E_u_delta'],
    controller=c['I_v_delta'],
    rate_law=beta*S_u*(Mud*I_v_delta) / N
)
# Sv -> Evd by Ivd
svvd = ControlledConversion(
    subject=c['S_v'],
    outcome=c['E_v_delta'],
    controller=c['I_v_delta'],
    rate_law=beta*S_v*(Mvd*I_v_delta) / N
)


#### E -> I
# Eua -> Iua
euaiua = NaturalConversion(
    subject=c['E_u_alpha'],
    outcome=c['I_u_alpha'],
    rate_law=r_EI*E_u_alpha
)
# Eva -> Iva
evaiva = NaturalConversion(
    subject=c['E_v_alpha'],
    outcome=c['I_v_alpha'],
    rate_law=r_EI*E_v_alpha
)
# Eud -> Iud
eudiud = NaturalConversion(
    subject=c['E_u_delta'],
    outcome=c['I_u_delta'],
    rate_law=r_EI*E_u_delta
)
# Evd -> Ivd
evdivd = NaturalConversion(
    subject=c['E_v_delta'],
    outcome=c['I_v_delta'],
    rate_law=r_EI*E_v_delta
)

#### I -> R
# Iua -> Ru
iuaru = NaturalConversion(
    subject=c['I_u_alpha'],
    outcome=c['R_u'],
    rate_law=r_IR_ua*I_u_alpha
)
# Iva -> Rv
ivarv = NaturalConversion(
    subject=c['I_v_alpha'],
    outcome=c['R_v'],
    rate_law=r_IR_va*I_v_alpha
)
# Iud -> Ru
iudru = NaturalConversion(
    subject=c['I_u_delta'],
    outcome=c['R_u'],
    rate_law=r_IR_ud*I_u_delta
)
# Ivd -> Rv
ivdrv = NaturalConversion(
    subject=c['I_v_delta'],
    outcome=c['R_v'],
    rate_law=r_IR_ud*I_v_delta
)

#### I -> H
# Iua -> Hu
iuahu = NaturalConversion(
    subject=c['I_u_alpha'],
    outcome=c['H_u'],
    rate_law=r_IH_ua*I_u_alpha
)
# Iva -> Hv
ivahv = NaturalConversion(
    subject=c['I_v_alpha'],
    outcome=c['H_v'],
    rate_law=r_IH_va*I_v_alpha
)
# Iud -> Hu
iudhu = NaturalConversion(
    subject=c['I_u_delta'],
    outcome=c['H_u'],
    rate_law=r_IH_ud*I_u_delta
)
# Ivd -> Hv
ivdhv = NaturalConversion(
    subject=c['I_v_delta'],
    outcome=c['H_v'],
    rate_law=r_IH_ud*I_v_delta
)


#### H -> R
# Hu -> Ru
huru = NaturalConversion(
    subject=c['H_u'],
    outcome=c['R_u'],
    rate_law=r_HR_u*H_u
)
# Hv -> Rv
hvrv = NaturalConversion(
    subject=c['H_v'],
    outcome=c['R_v'],
    rate_law=r_HR_v*H_v
)


#### H -> D
# Hu -> Du
hudu = NaturalConversion(
    subject=c['H_u'],
    outcome=c['D_u'],
    rate_law=r_HD_u*H_u
)
# Hv -> Dv
hvdv = NaturalConversion(
    subject=c['H_v'],
    outcome=c['D_v'],
    rate_law=r_HD_v*H_v
)


### Cumulative Cases
# Eua -> Iua
ccua = ControlledProduction(
    controller=c['E_u_alpha'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_u_alpha
)
# Eva -> Iva
ccva = ControlledProduction(
    controller=c['E_v_alpha'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_v_alpha
)
# Eud -> Iud
ccud = ControlledProduction(
    controller=c['E_u_delta'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_u_delta
)
# Evd -> Ivd
ccvd = ControlledProduction(
    controller=c['E_v_delta'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E_v_delta
)


### Cumulative Hospitalizations
# Iua -> Hu
chua = NaturalConversion(
    subject=c['I_u_alpha'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_ua*I_u_alpha
)
# Iva -> Hv
chva = NaturalConversion(
    subject=c['I_v_alpha'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_va*I_v_alpha
)
# Iud -> Hu
chud = NaturalConversion(
    subject=c['I_u_delta'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_ud*I_u_delta
)
# Ivd -> Hu
chvd = NaturalConversion(
    subject=c['I_v_delta'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IH_vd*I_v_delta
)

### (7) Define observables

In [16]:
observables_seir = {
    'susceptible': Observable(name='susceptible', expression=S_u+S_v),
    'exposed': Observable(name='exposed', expression=E_u_alpha+E_v_alpha+E_u_delta+E_v_delta),
    'infected': Observable(name='infected', expression=I_u_alpha+I_v_alpha+I_u_delta+I_v_delta),
    'recovered': Observable(name='recovered', expression=R_u+R_v),
    'hospitalized': Observable(name='hospitalized', expression=H_u+H_v),
    'deceased': Observable(name='deceased', expression=D_u+D_v),
    'all_cases': Observable(name='all_cases', expression=Cumulative_cases),
    'all_hosp': Observable(name='all_hosp', expression=Cumulative_hosp)
}

### (8) Define template model and save as a petrinet AMR

In [17]:
seir_model = TemplateModel(
    templates=[
        suua,
        svua,
        suva,
        svva,
        suud,
        svud,
        suvd,
        svvd,
        euaiua,
        evaiva,
        eudiud,
        evdivd,
        iuaru,
        ivarv,
        iudru,
        ivdrv,
        iuahu,
        ivahv,
        iudhu,
        ivdhv,
        huru,
        hvrv,
        hudu,
        hvdv,
        ccua,
        ccva,
        ccud,
        ccvd,
        chua,
        chva,
        chud,
        chvd
    ],
    parameters=parameters,
    initials=initials,
    time=Time(name='t', units=day_units()),
    observables=observables_seir,
    annotations=Annotations(name='SEIRHD vacc var model')
)

# Save as JSON
with open("SEIRHD_vacc_var_petrinet.json", 'w') as fh:
    json.dump(template_model_to_petrinet_json(seir_model), fh, indent=1)

# Model 3: SEIRHD with NPI of Type 1

See https://www.mdpi.com/1660-4601/18/17/9027 for model derivation 

### (1) Define units (same as above)

### (2) Define model concepts

In [18]:
c = {
    "S": Concept(name="S", units=person_units(), identifiers={"ido": "0000514"}),
    "E": Concept(name="E", units=person_units(), identifiers={"apollosv": "0000154"}),
    "I": Concept(name="I", units=person_units(), identifiers={"ido": "0000511"}),
    "R": Concept(name="R", units=person_units(), identifiers={"ido": "0000592"}),
    "H": Concept(name="H", units=person_units(), identifiers={"ido": "0000511"}, 
            context={"property": "ncit:C25179"}),
    "D": Concept(name="D", units=person_units(), identifiers={"ncit": "C28554"}),
    "Cumulative_cases": Concept(name="Cumulative_cases", units=person_units()),
    "Cumulative_hosp": Concept(name="Cumulative_hosp", units=person_units())
}

for concept in c:
    c[concept].name = concept

### (3) Define parameters with uncertainty

In [19]:
parameters = {
    'beta': Parameter(name='kappa', value=sympy.Float(0.25), units=per_day_units(),
                      distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.2,
                                                            'maximum': 0.4})),  # Transmission rate
    'N': Parameter(name='total_population', value=sympy.Float(NYS_total_pop), units=person_units()),  # Total population
    
    'r_EI': Parameter(name='r_EI', value=sympy.Float(0.25), units=per_day_units(),
                     distribution=Distribution(type='StandardUniform1',
                                                parameters={'minimum': 0.2,
                                                            'maximum': 0.3})),  # Rate of progressing E -> I
    
    'r_IR': Parameter(name='r_IR', value=sympy.Float(0.2), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.15,
                                                        'maximum': 0.3})),  # Rate of progressing I -> R, H

    'p_IH': Parameter(name='p_IH', value=sympy.Float(0.1), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.05,
                                                        'maximum': 0.2})),  # Percent of I -> H
    
    'r_HR': Parameter(name='r_HR', value=sympy.Float(0.16), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.1,
                                                        'maximum': 0.2})),  # Rate of progressing H -> R = (1/avg_length_of_hospital_stay)
    
    'p_HD': Parameter(name='r_HD', value=sympy.Float(0.083), units=per_day_units(),
                 distribution=Distribution(type='StandardUniform1',
                                            parameters={'minimum': 0.08,
                                                        'maximum': 0.09})),  # Percent of H -> D
}

### (4) Define `SymPy` variables

In [20]:
S, E, I, R, H, D, Cumulative_cases, Cumulative_hosp, beta, N, r_EI, r_IR, p_IH, r_HR, p_HD = sympy.symbols(
    'S E I R H D Cumulative_cases Cumulative_hosp beta N r_EI r_IR p_IH r_HR p_HD')

### (5) Set initial conditions

In [21]:
initials = {
    "S": Initial(concept=c["S"], expression=NYS_total_pop - sum(all_initials)),
    "E": Initial(concept=c["E"], expression=all_initials[0]),
    "I": Initial(concept=c["I"], expression=all_initials[1]),
    "R": Initial(concept=c["R"], expression=all_initials[2]),
    'H': Initial(concept=c["H"], expression=all_initials[3]),
    'D': Initial(concept=c["D"], expression=all_initials[4]),
    "Cumulative_cases": Initial(concept=c["Cumulative_cases"], expression=cumul_cases0),
    "Cumulative_hosp": Initial(concept=c["Cumulative_hosp"], expression=cumul_hosp0)
}

### (6) Define templates

In [22]:
##### S -> E
se = ControlledConversion(
    subject=c['S'],
    outcome=c['E'],
    controller=c['I'],
    rate_law=beta*S*I / N
)


#### E -> I
ei = NaturalConversion(
    subject=c['E'],
    outcome=c['I'],
    rate_law=r_EI*E
)


#### I -> R
ir = NaturalConversion(
    subject=c['I'],
    outcome=c['R'],
    rate_law=r_IR*(1 - p_IH)*I
)


#### I -> H
ih = NaturalConversion(
    subject=c['I'],
    outcome=c['H'],
    rate_law=r_IR*p_IH*I
)


#### H -> R
hr = NaturalConversion(
    subject=c['H'],
    outcome=c['R'],
    rate_law=r_HR*(1 - p_HD)*H
)


#### H -> D
hd = NaturalConversion(
    subject=c['H'],
    outcome=c['D'],
    rate_law=r_HR*p_HD*H
)


### Cumulative Cases
# E -> I
ccei = ControlledProduction(
    controller=c['E'],
    outcome=c['Cumulative_cases'],
    rate_law=r_EI*E
)


### Cumulative Hospitalizations
# I -> H
chih = ControlledProduction(
    controller=c['I'],
    outcome=c['Cumulative_hosp'],
    rate_law=r_IR*p_IH*I
)

### (7) Define observables

In [23]:
observables_seir = {
    'susceptible': Observable(name='susceptible', expression=S),
    'exposed': Observable(name='exposed', expression=E),
    'infected': Observable(name='infected', expression=I),
    'recovered': Observable(name='recovered', expression=R),
    'hospitalized': Observable(name='hospitalized', expression=H),
    'deceased': Observable(name='deceased', expression=D),
    'all_cases': Observable(name='all_cases', expression=Cumulative_cases),
    'all_hosp': Observable(name='all_hosp', expression=Cumulative_hosp)
}

### (8) Define template model and save as petrinet AMR

In [24]:
seir_model = TemplateModel(
    templates=[
        se,
        ei,
        ir,
        ih,
        hr,
        hd,
        ccei,
        chih
    ],
    parameters=parameters,
    initials=initials,
    time=Time(name='t', units=day_units()),
    observables=observables_seir,
    annotations=Annotations(name='SEIRHD base model')
)

# Save as JSON
with open("SEIRHD_base_petrinet.json", 'w') as fh:
    json.dump(template_model_to_petrinet_json(seir_model), fh, indent=1)